
#### _ML продвинутые методы / ДЗ №1 / Практическая часть / Задача 2_

## 2. _LinearRegression vs Tree-based algorithms_ 

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
from sklearn.datasets import load_boston

In [4]:
data = load_boston()
X = data.data
y = data.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, random_state=241)

X_train.shape, X_test.shape

((406, 13), (100, 13))

In [6]:
cv = KFold(X_train.shape[0], shuffle=True, random_state=241)

In [7]:
%%time

lr = LinearRegression(n_jobs=-1)

print(-cross_val_score(lr, X_train, y_train, cv=cv, scoring='neg_mean_squared_error').mean())

23.103267524224805
Wall time: 431 ms


In [8]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [9]:
print('MSE (train): {:.3f}'.format(mean_squared_error(lr.predict(X_train), y_train)))
print('R2 (train): {:.3f}'.format(r2_score(lr.predict(X_train), y_train)))

MSE (train): 20.864
R2 (train): 0.674


In [10]:
print('MSE (test): {:.3f}'.format(mean_squared_error(lr.predict(X_test), y_test)))
print('R2 (test): {:.3f}'.format(r2_score(lr.predict(X_test), y_test)))

MSE (test): 27.780
R2 (test): 0.622


Если судить по **score** то качество предсказания линейной регрессии получается примерно на уровне `DecisionTreeRegressor`, но, конечно, проигрывает `RandomForestRegressor`.